In [2]:
!pip install scanpy
!pip install decoupler
!pip install leidenalg
!pip install omnipath

  Using cached anndata-0.10.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached h5py-3.11.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached natsort-8.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached numba-0.59.1-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached scipy-1.13.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached array_api_compat-1.6-py3-none-any.whl.metadata (1.4 kB)
     ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
     --------------------------------- ---- 143.4/162.8 kB 2.8 MB/s eta 0:00:01
     -------------------------------------- 162.8/162.8 kB 1.6 MB/s eta 0:00:00
  Using cached llvmlite-0.42.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached pytz-2024.1-py2.py3-

In [3]:
import anndata as ad
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import decoupler as dc

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

ModuleNotFoundError: No module named 'pydrive'